# **Road Following**

In this notebook, we will use the trained model to predict and control the **Jetracer** motion

In [ ]:
import sys
sys.path.append('../')
from IPython.display import display
import ipywidgets
import traitlets
import time
from camera import Camera, bgr8_to_jpeg

## **1. Creat JetRacer**

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [ ]:
'''Release the valve limit and control the speed completely by the interactive control'''
car.throttle_gain = 1
car.steering_gain = -1
'''Set the default offset of the front wheels'''
car.steering_offset = -0.037

#### 1.1 **Create an interactive slider to control Jetracer**

``throttle_slider`` Manually set the throttle size

``steering_slider`` Manually set the steering angle size

In [ ]:
throttle_switch = ipywidgets.FloatSlider(min=0, max=1, step=1, value=0) # Gaspedal (Pedal)
steering_slider = ipywidgets.FloatSlider(min=-0.8, max=0.8, step=0.1, value=0) # rotate
car_safe_button = ipywidgets.Button(description='SAFE',button_style='warning', layout=ipywidgets.Layout(width='300px', height='28px'))

def car_safe(change):
    car.throttle = 0
    car.steering = 0
    throttle_switch.value = 0
    steering_slider.value = 0

jetracer_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*24+' JetRacer control '+'-'*40),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('pause/start'),ipywidgets.Label('rotate')
        ]),
        ipywidgets.VBox([
            throttle_switch, steering_slider
        ])
    ]),
    car_safe_button
])

#display(jetracer_control_widgets)

#### 1.2 **Controls associated with JetRacer instances**

In [ ]:
#traitlets.dlink((throttle_slider, 'value'), (car, 'throttle'), transform=lambda x: x)
traitlets.dlink((steering_slider, 'value'), (car, 'steering'), transform=lambda x: x)
car_safe_button.on_click(car_safe)

## **2. Creating a Camera instance**
Set the image acquired by the camera to be a larger image, and later resize it to **resnet18** Supported inputs **224 X 224**

The size of the image acquired by the camera should apply to the input the camera is calibrated for (default is 608 * 342)

In [ ]:
image_width = 448#224
image_height = 448#224
display_width = 224
display_height = 224
camera_fps = 30

In [ ]:
camera = Camera(width=image_width, height=image_height, fps=camera_fps)

## **3. Load the model**

``target_number`` Set output target number

> Ensure that training has been completed and generate ``best_steering_model_xy.pth``

In [ ]:
model_path = 'model/best_follow_xy1_double_target_50epochs_trt.pth'
target_number = 2

## 3.2 **Load models**

In [ ]:
import torch
from torch2trt import TRTModule

model = TRTModule()
model.load_state_dict(torch.load(model_path))
device = torch.device('cuda')

## **4. Creation of pre-processing**

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

#### **Transforms data acquired from the camera into a format supported by the model.**

1. transform from HWC layout to CHW layout
2. remap from **[0, 255]** to **[0, 1]** using the same parameter normalization as in training
3. transfer data from **CPU** to **GPU**.
4. add batch size

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    #image = transforms.functional.resize(image,(224, 224))
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

## **5. Creation of control functions**
### **5.1 PID control of rotation**

In [ ]:
class SteeringPID(traitlets.HasTraits):
    Kp_bias = traitlets.Float()# Proportional gain bias
    Kp_factor = traitlets.Float()# Proportional gain factor
    Ki = traitlets.Float()# Integral gain
    Kd = traitlets.Float()# Derivative gain
    bias = traitlets.Float()# Bias gain

    def __init__(self, max_out, min_out, windup_guard):
        self.max_out = max_out # Maximum output of PID algorithm
        self.min_out = min_out # Minimum output of PID algorithm
        self.integral_error = 0 # Accumulated error
        self.last_error = 0 # Last error
        self.current_time = time.time() # Current time
        self.last_time = self.current_time
        self.setPoint = 0.0
        self.windup_guard = windup_guard # Maximum integral term
        self.new_Kp = 0.0

    def calculate(self, pv):
        '''Calculate PID based on input error'''
        error = self.setPoint - pv

        self.current_time = time.time()
        delta_time = self.current_time - self.last_time

        self.new_Kp = (error*error)*self.Kp_factor + self.Kp_bias
        P_out = error* self.new_Kp # Proportional term

        self.integral_error += error * delta_time
        if self.integral_error < -self.windup_guard:
            self.integral_error = -self.windup_guard
        elif self.integral_error > self.windup_guard:
            self.integral_error = self.windup_guard
        I_out = self.Ki * self.integral_error # Integral term

        derivative = (error - self.last_error) / delta_time # Derivative term
        D_out = self.Kd * derivative

        PID_out = P_out + I_out + D_out + self.bias # Calculate final PID output

        # Check upper and lower bounds
        if PID_out > self.max_out:
            PID_out = self.max_out
        elif PID_out < self.min_out:
            PID_out = self.min_out

        self.last_error = error # Save current error for next calculation
        self.last_time = self.current_time # Save last timestamp

        return PID_out

    def reset(self):
        self.integral_error = 0
        self.last_error = 0
        self.current_time = time.time() # Current time
        self.last_time = self.current_time
        self.setPoint = 0.0
        return 'success reset'
    def get_value(self):
        return self.new_Kp, self.Ki, self.Kd

'''Instantiate PID controller and set maximum steering angle'''
steering_controller = SteeringPID(0.9, -0.9, 0.8)

### **5.2 Create Interactive Controls for PID Tuning**
``steering_kp_slider`` Sets the steering kP (proportional gain), which has a linear relationship with``error``A larger value means faster convergence to the target position (faster error reduction), but setting it too high may cause oscillations around the target.

``steering_kd_slider`` Sets the steering kD (derivative gain), which responds to the rate of error change (derivative of error, representing speed). This parameter helps dampen oscillations caused by the P term.

``steering_bias_slider`` Sets the bias term. If the robot consistently drifts to one side during operation, adjust this parameter to keep the JetRacer centered.

In [ ]:
import time

steering_Kp_bias_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.32) # Proportional gain - bias term
steering_Kp_factor_slider = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=1.10) # Proportional gain - coefficient
steering_Ki_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0) # Integral gain
steering_Kd_slider = ipywidgets.FloatSlider(min=-0.6, max=0.6, step=0.01, value=0.02) # Derivative gain
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.00) # Bias gain
pid_reset_button = ipywidgets.Button(description='PID RESET',button_style='warning')

def pid_reset(change):
    global steering_controller
    steering_controller.reset()


steering_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*38+' Steering PID Control Parameters '+'-'*38),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('Kp(Proportional gain - bias term)'),ipywidgets.Label('Kp(Proportional gain - coefficient)'),
            ipywidgets.Label('Ki(Integral gain)'),
            ipywidgets.Label('Kd(Derivative gain)/10'),ipywidgets.Label('bias(Bias gain)'),
        ]),
        ipywidgets.VBox([
            steering_Kp_bias_slider, steering_Kp_factor_slider,
            steering_Ki_slider, steering_Kd_slider,
            steering_bias_slider
        ])
    ]),
    pid_reset_button
])


time.sleep(1)
traitlets.dlink((steering_Kp_bias_slider, 'value'), (steering_controller, 'Kp_bias'), transform=lambda x: x)
traitlets.dlink((steering_Kp_factor_slider, 'value'), (steering_controller, 'Kp_factor'), transform=lambda x: x)
traitlets.dlink((steering_Ki_slider, 'value'), (steering_controller, 'Ki'), transform=lambda x: x)
traitlets.dlink((steering_Kd_slider, 'value'), (steering_controller, 'Kd'), transform=lambda x: x/10)
traitlets.dlink((steering_bias_slider, 'value'), (steering_controller, 'bias'), transform=lambda x: x)
pid_reset_button.on_click(pid_reset)

### **5.3 Judging straight lines and curves**

In [ ]:
class SpeedControl(traitlets.HasTraits):
    straight_throttle = traitlets.Float()
    curve_throttle = traitlets.Float()
    angle_threshold = traitlets.Float()
    min_tracking_number = traitlets.Int()

    def __init__(self, max_out, min_out):
        self.errors = []
        self.curve_straight = 'curve'
        self.out = 0.0
        self.max_out = max_out
        self.min_out = min_out
    def calculate(self, angle):
        self.errors.append(angle)

        self.curve_straight = 'curve'
        self.out = self.curve_throttle

        if len(self.errors) > self.min_tracking_number:
            del(self.errors[0])

            if all(_ > -self.angle_threshold for _ in self.errors) and all(_ < self.angle_threshold for _ in self.errors):
                self.curve_straight = 'straight'
                self.out = self.straight_throttle

        if self.out > self.max_out:
            self.out = self.max_out
        elif self.out < self.min_out:
            self.out = self.min_out
        return self.out, self.curve_straight

    def reset(self):
        del(self.errors[:])
    def get_value(self):
        return self.straight_throttle, self.curve_throttle
    def get_threshold(self):
        return self.angle_threshold

throttle_controller = SpeedControl(0.25, 0.1)

### **5.4 Creating Interactive Controls for Speed Controllers**

In [ ]:
straight_throttle_sliber = ipywidgets.FloatSlider(min=0.10, max=0.25, step=0.01, value=0.18)
curve_throttle_sliber = ipywidgets.FloatSlider(min=0.10, max=0.25, step=0.01, value=0.14)
angle_threshold_sliber = ipywidgets.FloatSlider(min=0.01, max=0.9, step=0.02, value=0.1)
min_tracking_number_sliber = ipywidgets.IntSlider(min=0, max=50, step=1, value=10)

traitlets.dlink((straight_throttle_sliber, 'value'), (throttle_controller, 'straight_throttle'), transform=lambda x: x)
traitlets.dlink((curve_throttle_sliber, 'value'), (throttle_controller, 'curve_throttle'), transform=lambda x: x)
traitlets.dlink((angle_threshold_sliber, 'value'), (throttle_controller, 'angle_threshold'), transform=lambda x: x)
traitlets.dlink((min_tracking_number_sliber, 'value'), (throttle_controller, 'min_tracking_number'), transform=lambda x: x)

throttle_control_widgets = ipywidgets.VBox([
    ipywidgets.Label('-'*30+' Longitudinal Throttle Control Parameters '+'-'*30),
    ipywidgets.HBox([
        ipywidgets.VBox([
            ipywidgets.Label('Straight section max speed'),ipywidgets.Label('Curve section max speed'),
            ipywidgets.Label('Straight/curve judgment threshold'),ipywidgets.Label('Minimum stable tracking count')
        ]),
        ipywidgets.VBox([
            straight_throttle_sliber, curve_throttle_sliber, angle_threshold_sliber, min_tracking_number_sliber
        ])
    ])
])

## **6. Visualization of predicted results and operational status**

#### **6.1 Define interactive controls to display prediction results and running status**

``live_widget`` Display image data

``live_steering_slider`` Display JetRacer's current turning angle [-1, 1]

``live_throttle_slider`` Display JetRacer's current forward direction [-1, 1]

> Only display values, manual modification will not affect the values ​​of related variables in the program

In [ ]:
live_image_widget = ipywidgets.Image(format='jpeg', width=display_width, height=display_height)

live_steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0)
live_throttle_slider = ipywidgets.FloatSlider(min=0, max=1.0)

prediction_text = ipywidgets.Textarea()

live_widget = ipywidgets.VBox([
    ipywidgets.Label('-'*65+'  LIVE  '+'-'*65),
    ipywidgets.HBox([
        live_image_widget,
        ipywidgets.VBox([
            prediction_text,
            ipywidgets.HBox([
                ipywidgets.VBox([
                    ipywidgets.Label('live_steering'), ipywidgets.Label('live_throttle')
                ]),
                ipywidgets.VBox([
                    live_steering_slider,live_throttle_slider
                ])
            ])
        ])
    ])

])

#### **6.2 Creating Angle Threshold Drawing Functions**

In [ ]:
def draw_angle_threshold(image, threshold):
    h, w = image.shape[:2]
    origin_point = (int(w/2), h-1)
    if threshold < np.arctan2(w/2, h) and threshold > np.arctan2(-w/2, h):
        left_point = (int(w/2 + h*np.tan(-1*threshold)), 0)
        right_point = (int(w/2 + h * np.tan(threshold)), 0)
        image = cv2.line(image, origin_point, left_point, (0, 0, 255), 2)
        image = cv2.line(image, origin_point, right_point, (0, 0, 255), 2)
    else:
        left_point = (0, int(h + w / 2 / np.tan(-1*threshold)))
        right_point = (w-1, int(h - w / 2 / np.tan(threshold)))
        image = cv2.line(image, origin_point, left_point, (0, 255, 0), 2)
        image = cv2.line(image, right_point, origin_point, (0, 255, 0), 2)
    return image

#### **6.3 Define the drawing function function**

In [ ]:
def drawing(image, targets, fps, angle_threshold):
    h, w = image.shape[:2]

    image = cv2.line(image, (int(w/2), 0), (int(w/2), h-1), (20, 150, 20),2)
    image = cv2.line(image, (0, int(h/2)), (w-1, int(h/2)), (20, 150, 20),2)

    image = draw_angle_threshold(image, angle_threshold)

    for target in targets:
        x, y, color = target
        circle_x = int(w * (x + 1.0)/2.0)
        circle_y = int(h * (y + 1.0)/2.0)
        image = cv2.circle(image, (circle_x, circle_y), 6, color, 2)

    #for index, text in enumerate(texts):
    image = cv2.putText(image, fps, (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    return image

## **7. Creation of the "processing" function**

#### **7.1 Define Processing Functions**

1. Preprocess camera image
2. Execute neural network
3. Calculate approximate steering value
4. Control motor using Proportional-Derivative (PD) control




In [ ]:
steering_controller.reset() # Initialize steering controller
throttle_controller.reset() # Initialize throttle controller

def execute(change):
    global Steering_controller, current_lane
    first_time = time.time()
    image = change['new']

    '''Original image calibration'''
    #image = correctImage(image,coefficient_group)
    t1 = time.time()

    '''Resize image'''
    image = cv2.resize(image, (224, 224))
    t2 = time.time()

    '''Model prediction'''
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    t3 = time.time()

    '''Get coordinates'''
    x_1, y_1 = xy[0], xy[1]
    x_2, y_2 = xy[2], xy[3]
    transform_y_1 = y_1/2.0 + 0.5 # Map to [0:1]
    transform_y_2 = y_2/2.0 + 0.5

    '''Calculate steering value'''
    # arctan2 returns the angle between target point and longitudinal centerline
    if current_lane == 'left':
        #angle = np.arctan2(x_1, transform_y_1)
        deviation = x_1
    elif current_lane == 'right':
        #angle = np.arctan2(x_2, transform_y_2)
        deviation = x_2
    pred_steering = -1 * steering_controller.calculate(deviation)

    end_time = time.time()

    '''Calculate throttle value based on road type'''
    if current_lane == 'left':
        transform_y = y_1 + 1.0
        angle = np.arctan2(x_1, transform_y)
    elif current_lane == 'right':
        transform_y = y_2 + 1.0
        angle = np.arctan2(x_2, transform_y)
    # Return throttle value and road type
    pred_throttle, curve_straight = throttle_controller.calculate(angle)

    '''Output control signals'''
    car.steering = pred_steering
    car.throttle = pred_throttle * throttle_switch.value

    end_time = time.time()

    '''Visualize detection results and runtime status'''
    # Draw target points and FPS on image
    targets = [
        [x_1, y_1, (0, 0, 255) if current_lane == 'left' else (0, 255, 0)],
        [x_2, y_2,(0, 0, 255) if current_lane == 'right' else (0, 255, 0)]
    ]
    image = drawing(image, targets, 'FPS: %.2f'%(1/(end_time-first_time)), throttle_controller.get_threshold())

    # Display information in text box

    texts = [
        'FPS: %.2f'%(1/(end_time-first_time)), # Show FPS
        'target_1: (%.2f, %.2f) %s'%(x_1, y_1, 'follow' if current_lane == 'left' else ''), # Show first target coordinates
        'target_2: (%.2f, %.2f) %s'%(x_2, y_2, 'follow' if current_lane == 'right' else ''), # Show second target coordinates
        'steering: Kp:%.3f Ki:%.3f Kd:%.3f'%(steering_controller.get_value()),
        'throttle: max: %.2f min: %.2f'%(throttle_controller.get_value()),
        'pred_steering: %.2f'%(pred_steering),
        'pred_throttle: %.2f'%(pred_throttle),
        'state: %s %s'%(curve_straight, 'Speed up'if curve_straight == 'straight' else 'slow down')
    ]
    new_text = ''
    for text in texts:
        new_text += text
        new_text += '\n'
    prediction_text.value = new_text
    # Show results in slider
    live_steering_slider.value = pred_steering
    #live_throttle_slider.value = pred_throttle

    live_image_widget.value = bgr8_to_jpeg(image)

#camera.observe(execute, names='value')
execute({'new': camera.value}) # Test run once

#### **7.2 Defining the Start Button**

In [ ]:
start_button = ipywidgets.Button(description='START',button_style='danger', layout=ipywidgets.Layout(width='300px', height='28px'))
start_widget = ipywidgets.VBox([
    ipywidgets.Label('-'*50+' START RUN '+'-'*50),
    start_button
])

def start_execute(change):
    global steering_controller, steering_controller
    if start_button.description == 'START':
        steering_controller.reset()
        throttle_controller.reset()
        camera.observe(execute, names='value')
        start_button.description = 'STOP'
        start_button.button_style='warning'
    else:
        camera.unobserve(execute, names='value')
        start_button.description = 'START'
        start_button.button_style='danger'

start_button.on_click(start_execute)

## **8.  Visualization of Detection Results and Parameter Tuning Module**
This section allows for control parameter tuning and validation of model prediction performance.

| Component | Function |
|--------|-------------|
| Throttle | Controls JetRacer's longitudinal movement |
| Steering | Controls JetRacer's lateral movement |
| SAFE | JetRacer homing (emergency stop) |
| Kp(Proportional gain) | Controls PID proportional gain |
| Ki(Integral gain) | Controls PID integral gain |
| Kd(Derivative gain) | Controls PID derivative gain |
| bias(Bias gain) | Controls PID bias gain |
| live_steering | Displays current steering angle |
| live_throttle | Displays current throttle value |
| LEFT/RIGHT | Controls lane changing |
| START | Movement start switch |

In [ ]:
display(
    ipywidgets.VBox([
        ipywidgets.HBox([
            ipywidgets.VBox([jetracer_control_widgets,throttle_control_widgets]),
            steering_control_widgets
        ]),
        live_widget,
        change_lane_widgets,
        start_widget
    ])
)

## **9. release Camera and JetRacer**

In [ ]:
#car.stop()
camera.stop()